In [ ]:
#Necessary Libraries
import numpy as np
import tensorflow
from keras.utils import np_utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, ResNet50, Xception

In [ ]:
#Mounting the drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Unzipping the nature_12K.zip dataset
zip_path = "/content/drive/MyDrive/Dataset/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [ ]:
#Installing the wandb library

!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 45.2 MB/s 
     |████████████████████████████████| 144 kB 46.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [ ]:
# Storing the training and testing directories
import os 
Tags = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
no_class = 10
train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'

In [ ]:
# Generating the dataset for training and validation
def train_val__test_data_generation(image_size=128,batch_size=256,augment_data=False):
    #Augmenting the data to avoid overfitting
    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=45,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=False)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    #Generating data batch by batch in order to make the model faster to run the dataset
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size, subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size, subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size,image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [ ]:
#Configurations to find out the best hyperparameters out of them
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'pre_trained_model': {
            'values':['Inceptionv3','Inceptionresnetv2','Xception','Resnet50']
        },
        'dropout': {
            'values': [0, 0.2, 0.3, 0.4]
        },
        'batch_norm':{
            'values': [True,False]
        },
        'augment_data': {
            'values': [True,False]
        },
        'batch_size': {
            'values': [32, 64, 128, 256]
        },
        'dense_size':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [ ]:
def PreTrainedModel():
    #default configurations
    config_defaults = {
        'weight_decay': 0.005,
        'dropout': 0.2,
        'batch_size': 64,
        'epochs': 10,
        'batch_norm': True,
        'augment_data': True,
        'dense_size': 256,
        'seed': 1234,
        'pre_trained_model': 'Xception'
    }
    image_size=128
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    #config will store the hyperparameters
    config = wandb.config
    wandb.run.name = 'pmodel_'+ str(config.pre_trained_model)+'dense_size_'+ str(config.dense_size)+'_dropout_'+str(config.dropout)+'_bn_'+str(config.batch_norm)+'_ag_'+ str(config.augment_data)

    if config.pre_trained_model == 'Inceptionv3':
        image_size=299
        pmodel = InceptionV3(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Inceptionresnetv2':
        image_size=299
        pmodel = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Xception':
        image_size=299
        pmodel = Xception(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    elif config.pre_trained_model == 'Resnet50':
        image_size=224
        pmodel = ResNet50(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
  

    pmodel.trainable = False
    model = Sequential()
    model.add(pmodel)
    model.add(Flatten())
    if config.batch_norm:
        model.add(BatchNormalization())
    model.add(Dense(config.dense_size, activation='relu'))
    model.add(Dropout(config.dropout))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    train_generator, val_generator, test_generator = train_val__test_data_generation(image_size=image_size,batch_size=config.batch_size,augment_data=config.augment_data)
    hist=model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])
    val_acc=max(hist.history['val_accuracy'])
    params={'batch_norm':config.batch_norm,'augmentation':config.augment_data,'dropout':config.dropout,
            'pre_trained_model':config.pre_trained_model,'val_acc':val_acc}
    wandb.log(params)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910_DL_Assignment2", entity="nomads")


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 5uprwamt
Sweep URL: https://wandb.ai/nomads/CS6910_DL_Assignment2/sweeps/5uprwamt


In [ ]:
wandb.agent('i1haqmcw',PreTrainedModel, count = 1)

wandb: Agent Starting Run: mpc5ye1j with config:
wandb: 	augment_data: False
wandb: 	batch_norm: False
wandb: 	batch_size: 128
wandb: 	dense_size: 256
wandb: 	dropout: 0.4
wandb: 	pre_trained_model: Xception
wandb: 	weight_decay: 0
wandb: Currently logged in as: nomads (use `wandb login --relogin` to force relogin)


83697664/83683744 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
71/71 [==============================] - 225s 3s/step - loss: 2.8041 - accuracy: 0.6046 - val_loss: 0.8746 - val_accuracy: 0.7497 - _timestamp: 1648829450.0000 - _runtime: 240.0000
Epoch 2/10
71/71 [==============================] - 200s 3s/step - loss: 0.8396 - accuracy: 0.7408 - val_loss: 0.8453 - val_accuracy: 0.7648 - _timestamp: 1648829691.0000 - _runtime: 481.0000
Epoch 3/10
71/71 [==============================] - 194s 3s/step - loss: 0.6731 - accuracy: 0.7796 - val_loss: 0.7550 - val_accuracy: 0.7828 - _timestamp: 1648829885.0000 - _runtime: 675.0000
Epoch 4/10
71/71 [==============================] - 199s 3s/step - loss: 0.6263 - accuracy: 0.7968 - val_loss: 0.7213 - val_accuracy: 0.7938 - _timestamp: 1648830082.0000 - _runtime: 872.0000
Epoch 5/10
71/71 [=====================

accuracy,▁▅▆▆▇▇▇███
augmentation,▁
batch_norm,▁
dropout,▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▂▂▂▁▁▁▁▁
val_acc,▁
val_accuracy,▁▃▅▆▅▇▇▇▇█
val_loss,█▇▃▁▅▂▆▄▆▃
accuracy,0.867
augmentation,False


In [ ]:
#Testing the model with the best configuration on test data
def test():
    
    #best_configuration obtained from training and validation
        best_weight_decay= 0
        best_dropout= 0.4
        best_batch_size= 128
        best_dense_size= 256
        best_batch_norm= False
        best_data_augment= False
        best_pre_trained_model= 'Inceptionresnetv2'
        image_size=299
        
        pmodel=InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
        pmodel.trainable = False
        model = Sequential()
        model.add(pmodel)
        model.add(Flatten())
        if best_batch_norm:
            model.add(BatchNormalization())
        model.add(Dense(best_dense_size, activation='relu'))
        model.add(Dropout(best_dropout))
        model.add(Dense(10, activation='softmax'))
        model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
        train_generator, val_generator, test_generator = train_val__test_data_generation(image_size=image_size,batch_size=best_batch_size,augment_data=best_data_augment)
        model.fit(train_generator, epochs=10, validation_data=val_generator)

        return model

In [ ]:
model=test()
train_generator, val_generator, test_generator = train_val__test_data_generation(image_size=299,batch_size=128,augment_data=False)
model.evaluate(test_generator)

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
71/71 [==============================] - 237s 3s/step - loss: 2.5904 - accuracy: 0.6150 - val_loss: 0.8058 - val_accuracy: 0.7548
Epoch 2/10
71/71 [==============================] - 204s 3s/step - loss: 0.8630 - accuracy: 0.7390 - val_loss: 0.7411 - val_accuracy: 0.7698
Epoch 3/10
71/71 [==============================] - 204s 3s/step - loss: 0.7757 - accuracy: 0.7613 - val_loss: 0.7097 - val_accuracy: 0.7988
Epoch 4/10
71/71 [==============================] - 204s 3s/step - loss: 0.6706 - accuracy: 0.7816 - val_loss: 0.7296 - val_accuracy: 0.8178
Epoch 5/10
71/71 [==============================] - 204s 3s/step - loss: 0.6050 - accuracy: 0.7990 - val_loss: 0.6940 - val_accuracy: 0.8098
Epoch 6/10
71/71 [==============================] - 204s 3s/step - loss: 0.5538 - accuracy: 0.8157 - val_loss: 0.7344 - val_accuracy: 0.8198
Epoch 7/10
71/71 [========

[0.8794358372688293, 0.8169999718666077]